In [ ]:
from PIL import Image
import requests
url = "https://farm4.staticflickr.com/3487/3925656789_1b64654c91_z.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
from transformers import CLIPSegProcessor, CLIPSegForImageSegmentation

processor = CLIPSegProcessor.from_pretrained("CIDAS/clipseg-rd64-refined")
model = CLIPSegForImageSegmentation.from_pretrained("CIDAS/clipseg-rd64-refined")


In [ ]:
prompts = ["hat", "ball", "player", "red shirt"]
inputs = processor(
    text=prompts,
    images=[image] * len(prompts),
    padding="max_length",
    return_tensors="pt",
)


In [ ]:
outputs = model(**inputs)
preds = outputs.logits.unsqueeze(1)


In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots(1, 5, figsize=(15, 4))
[a.axis("off") for a in ax.flatten()]
ax[0].imshow(image)
[ax[i + 1].imshow(torch.sigmoid(preds[i][0])) for i in range(4)]
[ax[i + 1].text(0, -15, prompts[i]) for i in range(4)]


In [ ]:
url = "https://upload.wikimedia.org/wikipedia/en/1/1e/Baseball_%28crop%29.jpg"
prompt = Image.open(requests.get(url, stream=True).raw)


In [ ]:
encoded_image = processor(images=[image], return_tensors="pt")
encoded_prompt = processor(images=[prompt], return_tensors="pt")
outputs = model(**encoded_image, conditional_pixel_values=encoded_prompt.pixel_values)


In [ ]:
preds = outputs.logits.unsqueeze(1)
preds = torch.transpose(preds, 0, 1)
_, ax = plt.subplots(1, 2, figsize=(6, 4))
[a.axis("off") for a in ax.flatten()]
ax[0].imshow(image)
ax[1].imshow(torch.sigmoid(preds[0]))
